### Importing Modules

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import bokeh
from bokeh.plotting import figure, show
from bokeh.models import tickers, ranges
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Synthetic Data & Model

$\hspace{80 mm}$ Poisson distribution:

\begin{align}
\large
P(Y = y) = \dfrac{\lambda^{y}}{y}{\rm e}^{-\lambda}
\end{align}

$\hspace{80 mm}$ Will model lambda as a regression function:

\begin{align}
\large
\lambda  = a + bx \\
\end{align}

$\hspace{80 mm}$ However, since we need $ a + bx > 0$, it is more usual to take:

\begin{align}
\large
\lambda  = {\rm e}^{a + bx} \\
\end{align}

Steps: <br>
- Model lambda as a regression function
- Sample 100 values from $ X_{i} \sim N(0,1)$
- Get 100 values from  $ \lambda_{i}  = a + bx_{i} $
- Sample 1 value from $ Y_{i} \sim Pois(\lambda_{i})$ for **each i** (i.e., $ \lambda_{i} $)

In [2]:
n = 100
a = 0.5
b = -1
x_i = np.random.normal(0, 1, n)
lambda_vals = np.exp(a + b*x_i)      # shape = 100
y_i = np.random.poisson(lambda_vals) # If size is None (default), a single value is returned if lam is a scalar. 
                                     #  Otherwise, np.array(lam).size samples are drawn. 

# Plotting
p = figure(toolbar_location= None, outline_line_color = 'black')
p.plot_width = 350
p.plot_height = 350
p.scatter(x = x_i, y = y_i, size=8, line_width = 1, line_color = 'black', fill_color = 'firebrick', legend_label="Data")
p.axis.axis_label = 'x'
p.yaxis.axis_label = 'y'
p.legend.border_line_color = "black"
p.legend.border_line_alpha = 1
p.legend.label_text_color = 'black'
show(p)    

### Estimating a and b via maximum likelihood

In [3]:
def get_J_and_L(x_arr, y_arr, a, b, n):
    """
    Computes J matrix and L vector for a Poisson Distribution
    where lambda is model as:
    
    \begin{equation}
    \lambda  = {\rm e}^{a + bx}
    \end{equation}       
    """
    z = np.sum(x_arr*y_arr)

    # Get derivatives for NR
    dL_da = n*np.mean(y_arr) - np.exp(a)*np.sum(np.exp(b*x_arr))
    dL_db = z - np.exp(a)*np.sum(x_arr*np.exp(b*x_arr))

    # Second Partial Derivatives
    dL_da2 = - np.exp(a)*np.sum(np.exp(b*x_arr))
    dL_db2 = - np.exp(a)*np.sum((x_arr**2)*np.exp(b*x_arr))
    dL_dadb = - np.exp(a)*np.sum(x_arr*np.exp(b*x_arr))
    
    J = np.array([[dL_da2,  dL_dadb],
              [dL_dadb, dL_db2]])
    
    L_prime = np.array([[dL_da],[dL_db]])
    return J, L_prime

J Matrix and L vector

$$
J(a,b)=\begin{pmatrix}
    \dfrac{\partial^2 L}{\partial a^2}   & \dfrac{\partial^2 L}{\partial a \hspace{1mm} \partial b}\\
    \dfrac{\partial^2 L}{\partial a \hspace{1mm} \partial b}   & \dfrac{\partial^2 L}{\partial b^2}
\end{pmatrix} \\
\\
\text{}\\
L'(a,b) = (\dfrac{\partial L}{\partial a}, \dfrac{\partial L}{\partial b})
$$

In [4]:
J, L_prime = get_J_and_L(x_i, y_i, a, b, n)
print("J matrix:\n", J)
print("\nL_prime vector:\n", L_prime)

J matrix:
 [[-232.49608889  199.35981165]
 [ 199.35981165 -391.48323114]]

L_prime vector:
 [[ 5.50391111]
 [-1.96777016]]


### Newton Raphson Algorithm

In [5]:
def newton_n_iter(x, y, a_o, b_o, n_iter =10, output_message = False):
    """
    Performs Newton-Raphson for a definite number of iteration
    Args:
        guess (float): initial value for parameter
        tolerance (float): tolerance
    
    """
    #Initialize
    iter_number = 0
    status_message = 'Starting with Guess = ' + str(a_o) + "," + str(b_o) + '\n'
    
    a = [a_o]
    b = [b_o]
    for i in range(n_iter):
    
        J, L = get_J_and_L(x_i, y_i, a_o, b_o, n)
   
        a_1 ,b_1 = np.array([[a_o],[b_o]]) - np.linalg.inv(J) @ L
        a.append(a_1[0])
        b.append(b_1[0])
        
        # Update Values
        a_o, b_o = a_1[0], b_1[0]
    
        iter_number += 1
        status_message += 'Iteration #' + str(iter_number) + ':= ' + str(a_1[0]) + "," + str(b_1[0])+ '\n'
        
    status_message += 'Total No. of Iterations = '  +  str(iter_number)
    
    if output_message:
        return a_o, b_o, status_message, a, b
    return a_o, b_o, a, b

In [6]:
(a_o, b_o, status_message, a_array, b_array) = newton_n_iter(x_i, y_i, 1.6, 2, n_iter =20, output_message = True)
print(status_message)

Starting with Guess = 1.6,2
Iteration #1:= 0.9217916183500717,1.8810259455702836
Iteration #2:= 0.6543814364737615,1.6012772839986091
Iteration #3:= 0.9601737567437247,1.0542967116618984
Iteration #4:= 1.4273300378987681,0.27256043688341347
Iteration #5:= 1.2632771048466949,-0.3605562721822555
Iteration #6:= 0.856018287051497,-0.7587361358916912
Iteration #7:= 0.5949993466085645,-0.9493265104130607
Iteration #8:= 0.5359680945138017,-0.986510607350241
Iteration #9:= 0.5337625869656893,-0.9878314916057706
Iteration #10:= 0.5337597252563666,-0.9878331900529815
Iteration #11:= 0.5337597252515717,-0.9878331900558264
Iteration #12:= 0.533759725251572,-0.9878331900558263
Iteration #13:= 0.533759725251572,-0.9878331900558263
Iteration #14:= 0.533759725251572,-0.9878331900558263
Iteration #15:= 0.533759725251572,-0.9878331900558263
Iteration #16:= 0.533759725251572,-0.9878331900558263
Iteration #17:= 0.533759725251572,-0.9878331900558263
Iteration #18:= 0.533759725251572,-0.9878331900558263
Ite

In [7]:
# Plotting
p = figure(toolbar_location= None, outline_line_color = 'black')
p.plot_width = 350
p.plot_height = 350
p.line(x = range(21), y = a_array, line_width = 2, line_color = 'firebrick', legend_label="a")
p.line(x = range(21), y = b_array, line_width = 2, line_color = 'green', legend_label="b")
p.axis.axis_label = 'Iteration #'
p.yaxis.axis_label = 'a and b'
p.legend.border_line_color = "black"
p.legend.border_line_alpha = 1
p.legend.label_text_color = 'black'
p.legend.location = 'top_right'
show(p)  

### Estimators Distributions

$$
\large
Cov\hspace{1mm}\hat\theta = J^{-1}(\theta)\hspace{2mm}CovL'(\theta)\hspace{2mm}J^{-1}(\theta)
$$

$\hspace{80 mm}$ We estimate $ J(\theta)$ by $J(\hat\theta)$

$$
L'(a,b) = (\dfrac{\partial L}{\partial a}, \dfrac{\partial L}{\partial b})
$$

$$
Cov L'(a,b)=\begin{pmatrix}
    Var(\dfrac{\partial L}{\partial a})   & Cov(\dfrac{\partial L}{\partial a}, \dfrac{\partial L}{\partial b})\\
    Cov(\dfrac{\partial L}{\partial a}, \dfrac{\partial L}{\partial b})   & Var(\dfrac{\partial L}{\partial b})\\
    \end{pmatrix}
     =\begin{pmatrix}     
     \Sigma_{i=1}^{n}\lambda_i   & \Sigma_{i=1}^{n}x_i\lambda_i\\
     \Sigma_{i=1}^{n}x_i\lambda_i   & \Sigma_{i=1}^{n}x_i^2\lambda_i
\end{pmatrix} \\
\\
$$

Compute J matrix from best-estimators

In [8]:
print(a_o)
print(b_o)

0.533759725251572
-0.9878331900558263


True covariance matric using original parameters

In [9]:
J, L_prime = get_J_and_L(x_i, y_i, a, b, n)
print("J matrix:\n", J)
print("\nL vector:\n", L_prime)

J matrix:
 [[-232.49608889  199.35981165]
 [ 199.35981165 -391.48323114]]

L vector:
 [[ 5.50391111]
 [-1.96777016]]


In [10]:
def get_COV_L_prime(x_arr, y_arr, a, b, n):
    """
    Computes the covariance matrix of L'   
    """

    # Get derivatives for NR
    dL_da = np.sum(np.exp(x_arr + b*x_arr))
    dL_db = np.sum((x_arr**2) * np.exp(x_arr + b*x_arr))
    dL_dadb = np.sum(x_arr * np.exp(x_arr + b*x_arr))
    
    COV_L = np.array([[dL_da,  dL_dadb],
                      [dL_dadb, dL_db]])
    
    return COV_L

In [11]:
COV_L_prime = get_COV_L_prime(x_i, y_i, a, b, n)
COV_L_prime

array([[100.        ,  19.8132992 ],
       [ 19.8132992 , 115.07669151]])

In [12]:
COV_theta_hat = np.linalg.inv(J) @ COV_L_prime @ np.linalg.inv(J)
COV_theta_hat

array([[0.00874556, 0.00598294],
       [0.00598294, 0.00457643]])